## Компания по доставке продуктов на дом. Сервис доступен как в приложении на ios, так и на android. Есть фронтовая аналитика в AppMetrica, необходимо проанализировать поведение пользователей, а также оценить эффективность каналов их привлечения. 
 - MAU.
 - Количество установок в январе.
 - Присвойте пользователям когорты по дню установки приложения и посчитайте для них  конверсию из установки в покупку в течение 7 дней.
 - С какого платного маркетингового канала пришло больше всего новых пользователей?
 - Пользователи, пришедшие с каких каналов, показали самую низкую конверсию в первую покупку?
 - Пользователи, пришедшие с какого канала, имеют медианный первый чек выше?
 - Какой платный канал привлечения (среди рекламных) имеет самый высокий ROMI?


## Данные из AppMetrica за период с 1 января по 31 марта 2020, только по пользователям, зарегистрированным позднее 1 января 2020.

### Описание данных 

- __date__ – дата совершения события

- __event__ - событие

  __app_install__ – установка приложения\
  __app_start__ – открыть приложения\
  __register__ – зарегистрироваться \
  __search__ – перейти на страницу поиска товаров (каталог)\
  __open_item__ – открыть товар\
  __choose_item__ – отправить товар в корзину\
  __tap_basket__ – перейти в корзину\
  __purchase__ – подтверждение покупки   
  

- __gender__ – пол пользователя

- __os_name__ – платформа пользователя

- __city__ – город пользователя

- __device_id__ – идентификатор устройства пользователя

- __urm_source__ – канал, с которого пришел пользователь

  __yandex-direct__ – Яндекс директ\
  __google_ads__ – реклама в Google\
  __vk_ads__ – реклама в ВК\
  __instagram_ads__ – реклама в instagram\
  __facebook_ads__ – реклама в facebook\
  __referal__ – акция «приведи друга»\
  __‘-’__ - канал не определен или это скачивание приложения напрямую или посещение не с рекламы  
  
  
  
- purchase_sum – стоимость покупки (при совершении события ‘purchase’)

In [160]:
import pandas as pd

In [161]:
df = pd.read_csv('C:/Users/Константин/Downloads/KC_case_data .csv', parse_dates=['date'])
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads


In [162]:
df.dtypes

date            datetime64[ns]
event                   object
purchase_sum           float64
os_name                 object
device_id                int64
gender                  object
city                    object
utm_source              object
dtype: object

## Поиск MAU за все время

In [163]:
# Добавляю колонку с началом месяца
df['start_month'] = df['date'].astype("datetime64[M]")

In [164]:
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,start_month
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01-01
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01-01
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,2020-01-01
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,2020-01-01
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,2020-01-01


In [165]:
MAU = df.groupby('start_month', as_index=False)\
        .agg({'device_id': 'nunique'})\
        .rename(columns={'device_id':'AU'})

In [166]:
MAU

,start_month,AU
0,2020-01-01,99161
1,2020-02-01,75032
2,2020-03-01,74623


## Количество установок:

In [167]:
cnt_install = df.query('event=="app_install"')\
                .groupby('start_month', as_index=False)\
                .agg({'device_id': 'nunique'})\
                .rename(columns={'device_id':'count_installs'})

In [168]:
cnt_install.head()

,start_month,count_installs
0,2020-01-01,80297
1,2020-02-01,38078
2,2020-03-01,36222


## Присвойте пользователям когорты по дню установки приложения и посчитайте для них  конверсию из установки в покупку в течение 7 дней.

In [169]:
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,start_month
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01-01
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01-01
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,2020-01-01
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,2020-01-01
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,2020-01-01


In [170]:
# Ищу дату установки
install_date = df.query('event=="app_install"')\
                 .groupby('device_id', as_index=False)\
                 .agg({'date':'min'})\
                 .rename(columns={'date':'install_date'})

In [171]:
install_date.head()

,device_id,install_date
0,4013,2020-01-15
1,4016,2020-01-04
2,4046,2020-01-04
3,4065,2020-03-19
4,4067,2020-01-31


In [172]:
# Добавляю дату установки в df
full_df = df.merge(install_date, on = 'device_id', how = 'left')

In [173]:
full_df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,start_month,install_date
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01-01,2020-01-01
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01-01,2020-01-01
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,2020-01-01,NaT
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,2020-01-01,2020-01-01
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,2020-01-01,2020-01-01


In [174]:
# Ищу дату первой покупки
min_buy_date = full_df.query('event=="purchase"')\
                      .groupby('device_id', as_index=False)\
                      .agg({'date':'min'})\
                      .rename(columns={'date':'min_buy_date'})

In [175]:
min_buy_date.head()

,device_id,min_buy_date
0,4014,2020-02-01
1,4046,2020-01-04
2,4060,2020-03-07
3,4120,2020-01-07
4,4125,2020-01-07


In [176]:
# Добавляю дату первой покупки
full_df = full_df.merge(min_buy_date, on = 'device_id', how = 'left')

In [177]:
full_df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,start_month,install_date,min_buy_date
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01-01,2020-01-01,2020-01-07
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01-01,2020-01-01,2020-01-01
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,2020-01-01,NaT,2020-01-01
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,2020-01-01,2020-01-01,NaT
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,2020-01-01,2020-01-01,NaT


In [178]:
# Функция для подсчета разницы между установкой и первой покупкой
def day_diff(x, y):
 end_date = x. dt.to_period('D'). view (dtype='int64')
 start_date = y. dt.to_period('D'). view (dtype='int64')
 return end_date-start_date

In [179]:
# Считаю разницу между установкой и первой покупкой
full_df['diff_date'] = day_diff(full_df.min_buy_date , full_df.install_date )

In [180]:
full_df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,start_month,install_date,min_buy_date,diff_date
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01-01,2020-01-01,2020-01-07,6
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01-01,2020-01-01,2020-01-01,0
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,2020-01-01,NaT,2020-01-01,-9223372036854757546
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,2020-01-01,2020-01-01,NaT,9223372036854757546
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,2020-01-01,2020-01-01,NaT,9223372036854757546


In [181]:
# Отбираю необходимые столбцы
last_df = full_df[['device_id', 'install_date', 'min_buy_date', 'diff_date']]

In [182]:
last_df.head()

,device_id,install_date,min_buy_date,diff_date
0,669460,2020-01-01,2020-01-07,6
1,833621,2020-01-01,2020-01-01,0
2,1579237,NaT,2020-01-01,-9223372036854757546
3,1737182,2020-01-01,NaT,9223372036854757546
4,4029024,2020-01-01,NaT,9223372036854757546


In [183]:
# Убираю пользователей без даты установки
last_df = last_df.query('diff_date>=0')

In [184]:
last_df.head()

,device_id,install_date,min_buy_date,diff_date
0,669460,2020-01-01,2020-01-07,6
1,833621,2020-01-01,2020-01-01,0
3,1737182,2020-01-01,NaT,9223372036854757546
4,4029024,2020-01-01,NaT,9223372036854757546
5,5148476,2020-01-01,2020-01-24,23


In [185]:
# Удаляю дубликаты
exampl_df = last_df.drop_duplicates()

In [186]:
exampl_df.head()

,device_id,install_date,min_buy_date,diff_date
0,669460,2020-01-01,2020-01-07,6
1,833621,2020-01-01,2020-01-01,0
3,1737182,2020-01-01,NaT,9223372036854757546
4,4029024,2020-01-01,NaT,9223372036854757546
5,5148476,2020-01-01,2020-01-24,23


In [187]:
# Создаю колонку с сконвертированными пользователями
exampl_df['status_conv'] = exampl_df.diff_date <= 7

C:\Users\Константин\AppData\Local\Temp\ipykernel_16636\371097701.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exampl_df['status_conv'] = exampl_df.diff_date <= 7


In [188]:
exampl_df.head()

,device_id,install_date,min_buy_date,diff_date,status_conv
0,669460,2020-01-01,2020-01-07,6,True
1,833621,2020-01-01,2020-01-01,0,True
3,1737182,2020-01-01,NaT,9223372036854757546,False
4,4029024,2020-01-01,NaT,9223372036854757546,False
5,5148476,2020-01-01,2020-01-24,23,False


In [189]:
# Группирую сконвертировавшихся пользователей
conv_users =  exampl_df.query('status_conv')\
                       .groupby('install_date', as_index=False)\
                       .agg({'device_id':'count'})\
                       .rename(columns={'device_id':'count_conv_users'})

In [190]:
conv_users.head()

,install_date,count_conv_users
0,2020-01-01,1408
1,2020-01-02,1186
2,2020-01-03,834
3,2020-01-04,639
4,2020-01-05,587


In [191]:
# Группирую всех пользователей
all_users = exampl_df.groupby('install_date', as_index=False)\
                      .agg({'device_id':'count'})\
                      .rename(columns={'device_id':'count_all_users'})

In [192]:
all_users.head()

,install_date,count_all_users
0,2020-01-01,3579
1,2020-01-02,3144
2,2020-01-03,2402
3,2020-01-04,1831
4,2020-01-05,1671


In [193]:
# Объединяю таблицы
kagort_df_last = conv_users.merge(all_users, on = 'install_date')

In [194]:
kagort_df_last.head()

,install_date,count_conv_users,count_all_users
0,2020-01-01,1408,3579
1,2020-01-02,1186,3144
2,2020-01-03,834,2402
3,2020-01-04,639,1831
4,2020-01-05,587,1671


In [195]:
# Добавляю колонку с конверсией
kagort_df_last['conversation'] = kagort_df_last.count_conv_users/kagort_df_last.count_all_users

In [196]:
kagort_df_last.sort_values('conversation', ascending=False).head()

,install_date,count_conv_users,count_all_users,conversation
0,2020-01-01,1408,3579,0.393406
8,2020-01-09,558,1424,0.391854
14,2020-01-15,1650,4310,0.382831
13,2020-01-14,1973,5173,0.381403
1,2020-01-02,1186,3144,0.377226


In [197]:
kagort_df_last['conversation_rate'] = round(kagort_df_last.conversation * 100, 1)

In [198]:
kagort_df_last.head()

,install_date,count_conv_users,count_all_users,conversation,conversation_rate
0,2020-01-01,1408,3579,0.393406,39.3
1,2020-01-02,1186,3144,0.377226,37.7
2,2020-01-03,834,2402,0.347211,34.7
3,2020-01-04,639,1831,0.348990,34.9
4,2020-01-05,587,1671,0.351287,35.1


## С какого платного маркетингового канала пришло больше всего новых пользователей? 


In [199]:
df.groupby('utm_source', as_index=False)\
  .agg({'device_id':'nunique'})\
  .sort_values('device_id', ascending=False)\
  .rename(columns={'device_id':'count_users'})

,utm_source,count_users
0,-,52273
6,yandex-direct,40712
2,google_ads,38096
5,vk_ads,34086
3,instagram_ads,31048
1,facebook_ads,25959
4,referal,15926


## Проанализируйте на каком этапе воронки отваливается большая часть клиентов. Посмотрите отдельно сценарии для зарегистрированных и для незарегистрированных пользователей. На каком шаге отваливается больше всего зарегистрированных пользователей?

In [200]:
df_2 = df.query('event=="register"')

In [201]:
df_2 = df_2[['date', 'device_id']]

In [202]:
df_2.head()

,date,device_id
2669658,2020-01-01,294193
2669659,2020-01-01,22917617
2669660,2020-01-01,15248490
2669661,2020-01-01,252062
2669662,2020-01-01,2251583


In [203]:
df_2 = df_2.rename(columns={'date':'registr_date'})

In [204]:
# Добавляю колонку со статусом регистрации
df_3 = df.merge(df_2, on = 'device_id', how='left')

In [205]:
df_3.sort_values(['device_id', 'date']).head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,start_month,registr_date
408942,2020-01-15,app_start,NaN,android,4013,female,Saint-Petersburg,vk_ads,2020-01-01,NaT
420565,2020-01-15,choose_item,NaN,android,4013,female,Saint-Petersburg,vk_ads,2020-01-01,NaT
431643,2020-01-15,search,NaN,android,4013,female,Saint-Petersburg,vk_ads,2020-01-01,NaT
441968,2020-01-15,tap_basket,NaN,android,4013,female,Saint-Petersburg,vk_ads,2020-01-01,NaT
2563772,2020-01-15,app_install,NaN,android,4013,female,Saint-Petersburg,vk_ads,2020-01-01,NaT


In [206]:
df_3['registr_date'] = df_3['registr_date'].fillna (0)

In [207]:
df_3.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,start_month,registr_date
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01-01,2020-01-07 00:00:00
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01-01,2020-01-01 00:00:00
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,2020-01-01,2020-01-01 00:00:00
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,2020-01-01,2020-01-01 00:00:00
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,2020-01-01,0


### Произвожу расчеты для незарегестрированных пользователей

In [208]:
registration_users = df_3.query('registr_date==0')

In [209]:
registration_users.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,start_month,registr_date
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,2020-01-01,0
9,2020-01-01,app_start,NaN,ios,10274354,female,Saint-Petersburg,instagram_ads,2020-01-01,0
19,2020-01-01,app_start,NaN,ios,26876099,female,Moscow,google_ads,2020-01-01,0
21,2020-01-01,app_start,NaN,android,32967032,female,Moscow,yandex-direct,2020-01-01,0
23,2020-01-01,app_start,NaN,ios,1576132,male,Saint-Petersburg,vk_ads,2020-01-01,0


In [210]:
# Создаю таблицу с воронкой
registration_users_voronka  = pd.pivot_table(data = registration_users , 
                                             index = 'date',
                                             columns = 'event',
                                             values= 'device_id',
                                             aggfunc = 'nunique')

In [211]:
registration_users_voronka.head()

event,app_install,app_start,choose_item,search,tap_basket
date,,,,,
2020-01-01,1149,1457,938,1372,513
2020-01-02,1055,1400,933,1314,523
2020-01-03,897,1259,799,1185,459
2020-01-04,742,1092,662,1004,366
2020-01-05,680,1082,686,1018,390


In [212]:
cols_to_move = ['search', 'choose_item', 'tap_basket']

In [213]:
# Отбираю необходимые этапы воронки
registration_users_voronka_last = registration_users_voronka[cols_to_move]

In [214]:
registration_users_voronka_last.head()

event,search,choose_item,tap_basket
date,,,
2020-01-01,1372,938,513
2020-01-02,1314,933,523
2020-01-03,1185,799,459
2020-01-04,1004,662,366
2020-01-05,1018,686,390


In [215]:
# Считаю количество пользователей по воронке в процентах
registration_users_voronka_last['search_to_choose_item_ratio'] = round(registration_users_voronka_last.choose_item/registration_users_voronka_last.search, 3)
registration_users_voronka_last['choose_item_to_tap_basket'] = round(registration_users_voronka_last.tap_basket/registration_users_voronka_last.choose_item, 3)

C:\Users\Константин\AppData\Local\Temp\ipykernel_16636\1824676385.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  registration_users_voronka_last['search_to_choose_item_ratio'] = round(registration_users_voronka_last.choose_item/registration_users_voronka_last.search, 3)


In [216]:
registration_users_voronka_last.head()

event,search,choose_item,tap_basket,search_to_choose_item_ratio,choose_item_to_tap_basket
date,,,,,
2020-01-01,1372,938,513,0.684,0.547
2020-01-02,1314,933,523,0.710,0.561
2020-01-03,1185,799,459,0.674,0.574
2020-01-04,1004,662,366,0.659,0.553
2020-01-05,1018,686,390,0.674,0.569


### Произвожу расчеты для зарегестрированных пользователей

In [217]:
not_registration_users = df_3.query('registr_date!=0')

In [218]:
not_registration_users.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,start_month,registr_date
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01-01,2020-01-07 00:00:00
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01-01,2020-01-01 00:00:00
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,2020-01-01,2020-01-01 00:00:00
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,2020-01-01,2020-01-01 00:00:00
5,2020-01-01,app_start,NaN,android,5148476,male,Saint-Petersburg,-,2020-01-01,2020-01-24 00:00:00


In [219]:
# Создаю таблицу с воронкой
not_registration_users_voronka  = pd.pivot_table(data = not_registration_users , 
                                                 index = 'date',
                                                 columns = 'event',
                                                 values= 'device_id',
                                                 aggfunc = 'nunique')

In [220]:
not_registration_users_voronka.head()

event,app_install,app_start,choose_item,purchase,register,search,tap_basket
date,,,,,,,
2020-01-01,2430,2988,2308,907,1059,2871,1789
2020-01-02,2089,3701,2814,1031,1145,3508,2127
2020-01-03,1505,3885,2909,1049,1073,3683,2158
2020-01-04,1089,3832,2820,917,908,3637,2083
2020-01-05,991,4525,3385,1138,989,4308,2524


In [221]:
cols_to_move_2 = ['search', 'choose_item', 'tap_basket']

In [222]:
# Отбираю необходимые этапы
not_registration_users_voronka_last = not_registration_users_voronka[cols_to_move]

In [223]:
not_registration_users_voronka_last.head()

event,search,choose_item,tap_basket
date,,,
2020-01-01,2871,2308,1789
2020-01-02,3508,2814,2127
2020-01-03,3683,2909,2158
2020-01-04,3637,2820,2083
2020-01-05,4308,3385,2524


In [224]:
# Считаю количество пользователей по воронке в процентах
not_registration_users_voronka_last['search_to_choose_item_ratio'] = round(not_registration_users_voronka_last.choose_item/not_registration_users_voronka_last.search, 3)
not_registration_users_voronka_last['choose_item_to_tap_basket'] = round(not_registration_users_voronka_last.tap_basket/not_registration_users_voronka_last.choose_item, 3)

C:\Users\Константин\AppData\Local\Temp\ipykernel_16636\993955384.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_registration_users_voronka_last['search_to_choose_item_ratio'] = round(not_registration_users_voronka_last.choose_item/not_registration_users_voronka_last.search, 3)
C:\Users\Константин\AppData\Local\Temp\ipykernel_16636\993955384.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_registration_users_voronka_last['choose_item_to_tap_basket'] = round(not_registration_users_voronka_last

In [225]:
not_registration_users_voronka_last.head()

event,search,choose_item,tap_basket,search_to_choose_item_ratio,choose_item_to_tap_basket
date,,,,,
2020-01-01,2871,2308,1789,0.804,0.775
2020-01-02,3508,2814,2127,0.802,0.756
2020-01-03,3683,2909,2158,0.790,0.742
2020-01-04,3637,2820,2083,0.775,0.739
2020-01-05,4308,3385,2524,0.786,0.746


## Пользователи, пришедшие с каких каналов, показали самую низкую конверсию в первую покупку?

In [226]:
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,start_month
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01-01
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01-01
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,2020-01-01
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,2020-01-01
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,2020-01-01


In [227]:
users_with_buy =  df.query('event=="purchase"')\
                    .groupby('utm_source', as_index=False)\
                    .agg({'device_id':'nunique'})\
                    .rename(columns={'device_id':'count_users_buy'})

In [228]:
users_with_buy

,utm_source,count_users_buy
0,-,16598
1,facebook_ads,9017
2,google_ads,11339
3,instagram_ads,10762
4,referal,6362
5,vk_ads,12364
6,yandex-direct,12028


In [229]:
all_users = df.query('event=="app_start"')\
              .groupby('utm_source', as_index=False)\
              .agg({'device_id':'nunique'})\
              .rename(columns={'device_id':'count_all_users'})

In [230]:
all_users

,utm_source,count_all_users
0,-,52268
1,facebook_ads,25957
2,google_ads,38092
3,instagram_ads,31045
4,referal,15924
5,vk_ads,34079
6,yandex-direct,40707


In [231]:
full_df = all_users.merge(users_with_buy, on = 'utm_source')

In [232]:
full_df['CR'] = full_df.count_users_buy/full_df.count_all_users

In [233]:
full_df.sort_values('CR')

,utm_source,count_all_users,count_users_buy,CR
6,yandex-direct,40707,12028,0.295477
2,google_ads,38092,11339,0.297674
0,-,52268,16598,0.317556
3,instagram_ads,31045,10762,0.346658
1,facebook_ads,25957,9017,0.347382
5,vk_ads,34079,12364,0.362804
4,referal,15924,6362,0.399523


## Пользователи, пришедшие с какого канала, имеют медианный первый чек выше?

In [234]:
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,start_month
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01-01
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01-01
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,2020-01-01
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,2020-01-01
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,2020-01-01


In [235]:
# Ищу дату первой покупки
df_2 = df.query('event=="purchase"')\
         .groupby('device_id', as_index=False)\
         .agg({'date':'min'})\
         .rename(columns={'date':'first_buy'})

In [236]:
df_2.head()

,device_id,first_buy
0,4014,2020-02-01
1,4046,2020-01-04
2,4060,2020-03-07
3,4120,2020-01-07
4,4125,2020-01-07


In [237]:
df_3 = df.merge(df_2, on = 'device_id')

In [238]:
df_3.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,start_month,first_buy
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01-01,2020-01-07
1,2020-01-01,search,NaN,android,669460,female,Moscow,-,2020-01-01,2020-01-07
2,2020-01-03,app_start,NaN,android,669460,female,Moscow,-,2020-01-01,2020-01-07
3,2020-01-03,choose_item,NaN,android,669460,female,Moscow,-,2020-01-01,2020-01-07
4,2020-01-03,search,NaN,android,669460,female,Moscow,-,2020-01-01,2020-01-07


In [239]:
# Отбираю только строки с покупками
df_4 = df_3.dropna()

In [240]:
# Добавляю колонку со статусом первой покупки(true,false)
df_4['bol_first_buy'] = df_4.date==df_4.first_buy

C:\Users\Константин\AppData\Local\Temp\ipykernel_16636\2851948858.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_4['bol_first_buy'] = df_4.date==df_4.first_buy


In [241]:
df_4.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,start_month,first_buy,bol_first_buy
13,2020-01-07,purchase,269.0,android,669460,female,Moscow,-,2020-01-01,2020-01-07,True
22,2020-01-09,purchase,4687.5,android,669460,female,Moscow,-,2020-01-01,2020-01-07,False
50,2020-01-01,purchase,329.5,ios,833621,male,Moscow,vk_ads,2020-01-01,2020-01-01,True
62,2020-01-11,purchase,601.0,ios,833621,male,Moscow,referal,2020-01-01,2020-01-01,False
77,2020-01-16,purchase,277.5,ios,833621,male,Moscow,referal,2020-01-01,2020-01-01,False


In [242]:
# Отбираю только первые покупки
df_5 = df_4.query('bol_first_buy')

In [243]:
median_first_buy = df_5.groupby('utm_source', as_index=False)\
                       .agg({'purchase_sum':'median'})\
                       .sort_values('purchase_sum')

In [244]:
median_first_buy

,utm_source,purchase_sum
1,facebook_ads,389.0
2,google_ads,390.5
6,yandex-direct,392.5
5,vk_ads,393.0
3,instagram_ads,393.5
4,referal,395.5
0,-,398.5


## Какой платный канал привлечения (среди рекламных) имеет самый высокий ROMI? 

### Данные по затратам на рекламу:

 - __Яндекс – 10491707 руб.__

 - __Гугл – 10534878 руб.__

 - __Фейсбук – 8590498 руб.__

 - __Инстаграм – 8561626 руб.__

 - __ВК – 9553531 руб.__

 - __Расходы на реферальную программу: если пользователь приведет друга и последний совершит первую покупку, то оба получат по 100 рублей.__

In [245]:
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,start_month
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01-01
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01-01
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,2020-01-01
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,2020-01-01
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,2020-01-01


In [246]:
# Ищу выручку, которую принесли каналы привлечения
df.groupby('utm_source', as_index=False)\
  .agg({'purchase_sum':'sum'})

,utm_source,purchase_sum
0,-,21449749.5
1,facebook_ads,12249901.0
2,google_ads,12868276.0
3,instagram_ads,14546969.0
4,referal,8837044.5
5,vk_ads,16389652.5
6,yandex-direct,13915368.0


In [247]:
ROMI_facebook_ads = ((12249901.0 - 8590498)/8590498)*100
ROMI_facebook_ads

42.59826380263403

In [248]:
ROMI_google_ads = ((12868276.0 - 10534878)/10534878)*100
ROMI_google_ads

22.149264566708794

In [249]:
ROMI_instagram_ads = ((14546969.0 - 8561626)/8561626)*100
ROMI_instagram_ads

69.90895187432854

In [250]:
ROMI_vk_ads = ((16389652.5 - 9553531)/9553531)*100
ROMI_vk_ads

71.55596710786828

In [251]:
ROMI_yandex = ((13915368.0 - 10491707)/10491707)*100
ROMI_yandex

32.63206835646478

In [252]:
referal_df = df.query('utm_source=="referal"')

In [253]:
referal_df = referal_df.query('event=="purchase"')

In [254]:
# Затраты на реферальную программу
referal_df.device_id.nunique()*200

1272400

In [255]:
ROMI_referal = ((8837044.5 - 1272400)/1272400)*100
ROMI_referal

594.517801005973